In [1]:
import sys
sys.path.append( '../')
import utils
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import glob

 
Music21 v.4 is the last version that will support Python 2.
Please start using Python 3 instead.

Set music21.environment.UserSettings()['warnings'] = 0
to disable this message.

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:


# dimension de l'espace latent
z_dim=30
#BETA=1 <=> VAE
#BETA>1 <=> Beta-VAE
# paramètre hyper important et marque l'importance que tu donnes à la KL divergence
#je te conseilles de le tuner et de le mettre >200 pour voir le résultat (ca devrait pas trop changer)
BETA=1

learning_rate = 1e-4

BATCH_SIZE = 2**5
epochs=1
TIMESTEPS=50

#midi files for the training
midi_files = glob.glob('../Sounds_for_training/*.mid')[:2]


dataset_string,offset=utils.concat_dataset(midi_files)
print(1)
dataset_number,unique_dic=utils.create_dataset_number(dataset_string)
print(2)
dataset_number=np.hstack((dataset_number,offset.reshape(-1,1)))

#normalize the dataset
mean1=np.mean(dataset_number[:,0])
std1=np.std(dataset_number[:,0])
dataset_number[:,0]-=mean1
dataset_number[:,0]/=std1
mean2=np.mean(dataset_number[:,1])
std2=np.std(dataset_number[:,1])
dataset_number[:,1]-=mean2
dataset_number[:,1]/=std2

x_train=utils.get_x(dataset_number,TIMESTEPS)
print(3)
X = tf.placeholder("float", [None, x_train.shape[1],x_train.shape[2]])


def encoder(X):  
    x = tf.layers.conv1d(inputs=X, filters=16, kernel_size=3, padding='same',activation=tf.nn.elu)
    x = tf.layers.conv1d(inputs=x, filters=32, kernel_size=3, padding='same',activation=tf.nn.elu)
    x = tf.layers.conv1d(inputs=x, filters=64, kernel_size=3, padding='same',activation=tf.nn.elu)
    x = tf.layers.conv1d(inputs=x, filters=64, kernel_size=3, padding='same',activation=tf.nn.elu)
    x = tf.layers.conv1d(inputs=x, filters=64, kernel_size=3, padding='same',activation=tf.nn.elu)
    x = tf.layers.conv1d(inputs=x, filters=128, kernel_size=3, padding='same',activation=tf.nn.elu)
    x = tf.contrib.layers.flatten(x)
    x = tf.layers.dense(x,128,activation=tf.nn.elu)
    z_mean = tf.layers.dense(x, z_dim)
    z_std=tf.layers.dense(x, z_dim)
    return z_mean,z_std

def sample_eps(z_mean,z_std):
    eps=tf.random_normal(tf.shape(z_mean),mean=0.,stddev=1.)
    z=z_mean+tf.exp(z_std/2)*eps
    return z

def decoder(z):
    x = tf.layers.dense(z,TIMESTEPS)
    x = tf.reshape(x,[-1,TIMESTEPS,1])
    x = tf.layers.conv1d(inputs=x, filters=64, kernel_size=3, padding='same',activation=tf.nn.elu)
    x = tf.layers.conv1d(inputs=x, filters=64, kernel_size=3, padding='same',activation=tf.nn.elu)
    x = tf.layers.conv1d(inputs=x, filters=64, kernel_size=3, padding='same',activation=tf.nn.elu)
    x = tf.layers.conv1d(inputs=x, filters=64, kernel_size=3, padding='same',activation=tf.nn.elu)
    x = tf.layers.conv1d(inputs=x, filters=64, kernel_size=3, padding='same',activation=tf.nn.elu)
    x = tf.layers.conv1d(inputs=x, filters=2, kernel_size=3, padding='same')
    x_sigmoid=tf.nn.sigmoid(x)
    return x,x_sigmoid

#you get the zmean and zstd from the encoder
z_mean,z_std=encoder(X)

#you generated a normal distribution with the above parameters
z=sample_eps(z_mean,z_std)

#you get the outuput of the decoder
output,output_sigmoid=decoder(z)

#the KL divergence
cons_loss= 0.5 * tf.reduce_sum(tf.exp(z_std) + z_mean**2 - 1. - z_std, 1)

# X_flat= tf.contrib.layers.flatten(X)
#how accurate is the sequence (reconstruction loss)
recons_loss=tf.reduce_sum(tf.pow(tf.subtract(output, X),2))

#the total cost
cost = tf.reduce_mean(recons_loss+BETA*cons_loss)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
sess=tf.Session()
sess.run(tf.global_variables_initializer())
val_loss_list1 = []
val_loss_list2 = []
for epoch in range(epochs):
    for batch_i in range(0, x_train.shape[0], BATCH_SIZE):
        _,loss1,loss2 = sess.run([optimizer,cons_loss,recons_loss], feed_dict={X: x_train[batch_i:batch_i + BATCH_SIZE]})
    print('KL divergence :',np.mean(loss1),'Reconstruction loss :',np.mean(loss2))
    val_loss_list1.append(np.mean(loss1))
    val_loss_list2.append(np.mean(loss2))

1
2
3
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
('KL divergence :', 70.82048, 'Reconstruction loss :', 633.84357)


In [4]:
# you take a sample of normal distribution
lala=[]
for i in range(1):
    z_sample=np.random.rand(1,z_dim)
    #you run it trough the nn
    a=sess.run(output,feed_dict={z:z_sample})
    #you get the result
    music_generated=a[0]
    #unormalize the result
    music_generated[:,0]*=std1
    music_generated[:,0]+=mean1
    music_generated[:,1]*=std2
    #add a bit of time to the offset (the time between each note)
    music_generated[:,1]+=mean2+0.25
    lala.append(music_generated[:,0])
    #you transform the array generated by a midi file
    utils.array_to_midi(music_generated,unique_dic,name='test'+str(i))

[26.229437   0.4719603]
[19.221878   0.5052416]
[18.812029    0.41552275]
[21.763393    0.36014944]
[29.845272   0.4057522]
[26.641594    0.52460724]
[15.126456    0.50374454]
[22.11692     0.37216628]
[28.163786   0.4002056]
[22.067406    0.51460785]
[16.15759    0.4831846]
[23.478498   0.3750818]
[32.598045    0.37970662]
[30.623457    0.49103048]
[27.360445   0.5090097]
[24.574587    0.43242255]
[29.523064   0.3746205]
[35.527763   0.4427256]
[24.619087   0.5186462]
[20.433624    0.42447045]
[36.55079    0.3701914]
[30.708954    0.49382007]
[16.302134   0.4862096]
[28.575485   0.3550052]
[33.70575     0.41254994]
[22.840103   0.5472181]
[16.127728   0.4824698]
[24.659472    0.37360394]
[29.095861   0.3987469]
[26.542603    0.47349122]
[29.617695    0.47612032]
[27.2905     0.4581874]
[25.581549   0.4206904]
[29.093775    0.40402874]
[26.698135   0.4344398]
[23.158876    0.44391167]
[24.72708     0.44158298]
[21.758518    0.47669682]
[13.557128    0.47434893]
[19.308786    0.40924612